In [ ]:
import sys
import numpy as np
import awkward as ak
import uproot 
import matplotlib.pyplot as plt

In [3]:
sys.path.insert(0, '../../MATHUSLA_FastSim/')
import DetectorSimulation.Detector as Detector
import DetectorSimulation.Particle as Particle
import Helpers.functions as hp

sys.path.insert(0,'../../FastSim_Additions')
from Additions import initiate_detector

In [6]:
file = uproot.open('../../AllSimData/nue_0-1Gev_100kev_gntp.0.gst.root:gst;2')
data = file.arrays(library = 'ak')

In [9]:
event_layout = transpose_daughters(data['Ef'], data['pxf'], data['pyf'], data['pzf'], data['pdgf'])

In [4]:
detector_benchmark = initiate_detector()

In [10]:
detector_benchmark.config.decay_x_dim

100.0

In [12]:
detector_benchmark.config.decay_x_max

50.0

In [15]:
detector_benchmark.config.decay_z_dim

AttributeError: 'ParamCard' object has no attribute 'decay_z_dim'

In [18]:
occurred = []

for entry in data['pdgf']:
    for pid in entry:
        if pid not in occurred:
            occurred.append(pid)

NameError: name 'sort' is not defined

In [21]:
occurred

[-14, -13, -11, 11, 12, 13, 14, 22, 2112, 2212, 1000070140, 1000080160]

In [22]:
pid_mass = {11:5.11e-4, 12:0, 13: 0.1057, 22:0, 2212:0.938, 2112: 0.939, 14:0}

In [23]:
event_layout[0,0]

<Array [0.438, -0.00271, -0.382, 0.214, 11] type='5 * float64'>

In [24]:
trial = event_layout[0,0]

In [26]:
mass = pid_mass[int(abs(trial[4]))]
mass

0.000511

In [28]:
particle = Particle.Particle(np.array([0,0,0]), trial[:4], trial[4], mass)

In [30]:
detector_benchmark.new_particle_event(particle)

In [33]:
success = detector_benchmark.return_current_particle()

In [40]:
success.tracker_hits

[]

In [46]:
def format_det_particle(daughter):
    return [*daughter.particle.momentum,
            daughter.particle.pid,
            len(daughter.tracker_hits),
           daughter._visibility]

In [43]:
position = (0,120,70)

In [47]:
all_events = ak.ArrayBuilder()

for event in event_layout[:100]:
    per_event = []
    for daughter in event:
        detector_benchmark.clear_detector()
        try:
            mass = pid_mass[int(abs(daughter[4]))]
            particle = Particle.Particle(position, daughter[:4], daughter[4], mass)
            detector_benchmark.new_particle_event(particle)
            formatted = format_det_particle(detector_benchmark.return_current_particle())
            per_event.append(formatted)
        except KeyError:
            pass
    all_events.append(per_event)

all_events = all_events.snapshot()

In [50]:
all_events[1]

<Array [[0.212, -0.101, 0.00954, ..., 11, 2], ...] type='2 * var * float64'>